## 0. Before we begin....

### Reference
* https://github.com/BrainOmicsCourse/BrainOmics2024/blob/main/1_Day1/Compiled/0_AssembleAdata.html
* https://github.com/BrainOmicsCourse/BrainOmics2024/blob/main/1_Day1/Resources.md
* https://scbrainregulation.su.domains/
* https://www.cell.com/cell/fulltext/S0092-8674(21)00942-9?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS0092867421009429%3Fshowall%3Dtrue

## 1. Data Download

In [1]:
### Name of Example Files
rna_counts = 'GSE162170_rna_counts.tsv'
rna_meta = 'GSE162170_rna_cell_metadata.txt'

### Data Path
path = '../data/'
Id = 'Id0003'
count_file  = f"{path}/{Id}/{rna_counts}"
meta_file   = f"{path}/{Id}/{rna_meta}"
output_file = f"{path}/{Id}/1_AssembledAdata.h5ad"

In [2]:
### Download Data
!curl -o {rna_counts}.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE162nnn/GSE162170/suppl/GSE162170%5Frna%5Fcounts%2Etsv%2Egz
!curl -o {rna_meta}.gz https://ftp.ncbi.nlm.nih.gov/geo/series/GSE162nnn/GSE162170/suppl/GSE162170%5Frna%5Fcell%5Fmetadata%2Etxt%2Egz
# If the download code is unavailable,
# please visit https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE162170
# and download two files below:
#   GSE162170_rna_counts.tsv.gz
#   GSE162170_rna_cell_metadata.txt.gz
# and place them to the proper directory

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  133M  100  133M    0     0  8814k      0  0:00:15  0:00:14  0:00:01 11.1M7k      0  0:00:16  0:00:12  0:00:04 10.9M  0  8891k      0  0:00:15  0:00:15 --:--:-- 11.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2829k  100 2829k    0     0  1360k      0  0:00:02  0:00:02 --:--:-- 1360k


In [3]:
### Unzip the data
!gzip -d {rna_counts}.gz
!gzip -d {rna_meta}.gz

### Place the data
!mkdir -p {path}/{Id}
!mv {rna_counts} {count_file}
!mv {rna_meta} {meta_file}

## 2. Build Anndata
### 2.1 Preparation

In [4]:
### System
import os
import sys

### Data Analysing
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt

### Single Cell RNA Sequening (mostly from scverse)
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix, isspmatrix
import scanpy.external as sce

### Custom functions
sys.path.append('../HelperFunctions')
import Helper as fn

In [5]:
### File Check
print(os.path.exists(count_file))
print(os.path.exists(meta_file))

True
True


### 2.2 Read RNA 

In [13]:
### 'count_file' will be the main of anndata
adata = sc.read_csv(
    count_file,
    delimiter='\t',
    first_column_names=None,
    dtype='float32'
    ) 

In [14]:
### You can choose different strategies when the Anndata is a sparse matrix or not.
### From here, Anndata will be *always* the sparse matrix, and be compressed.
if isspmatrix(adata.X) == False:
    adata.X = csr_matrix(adata.X)
    print('Converted adata.X to', type(adata.X))

Converted adata.X to <class 'scipy.sparse._csr.csr_matrix'>


In [16]:
adata.var

""
hft_w20_p3_r1_AAACCCAAGCTGCGAA
hft_w20_p3_r1_AAACCCAAGGTAGTAT
hft_w20_p3_r1_AAACCCACAACTCCAA
hft_w20_p3_r1_AAACCCACATAGTCAC
hft_w20_p3_r1_AAACCCAGTACAGGTG
...
hft_w16_p7_r2_TTTGTTGCAGCACCCA
hft_w16_p7_r2_TTTGTTGCAGGCTACC
hft_w16_p7_r2_TTTGTTGGTCGCTTAA
hft_w16_p7_r2_TTTGTTGGTCGTACAT


In [17]:
adata.obs

""
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000212907
ENSG00000198886
ENSG00000198786
ENSG00000198695


You can see `adata.var` shows barcodes, while `adata.obs` shows genes.
* We are going to transpose the anndata to replace them
* `obs`(observation) will be barcodes
* `var`(variable) will be genes

In [18]:
### Anndata transpose
adata = adata.transpose()
adata

AnnData object with n_obs × n_vars = 57868 × 33355

In [19]:
adata.obs

""
hft_w20_p3_r1_AAACCCAAGCTGCGAA
hft_w20_p3_r1_AAACCCAAGGTAGTAT
hft_w20_p3_r1_AAACCCACAACTCCAA
hft_w20_p3_r1_AAACCCACATAGTCAC
hft_w20_p3_r1_AAACCCAGTACAGGTG
...
hft_w16_p7_r2_TTTGTTGCAGCACCCA
hft_w16_p7_r2_TTTGTTGCAGGCTACC
hft_w16_p7_r2_TTTGTTGGTCGCTTAA
hft_w16_p7_r2_TTTGTTGGTCGTACAT


### 2.3 Metadata

In [20]:
meta = pd.read_csv(
    meta_file,
    sep = '\t',
    index_col='Cell.ID'
    )

In [21]:
meta

,Sample.ID,Age,Tissue.ID,Sample.Type,Assay,Batch,seurat_clusters,RNA.Counts,RNA.Features,Percent.MT,...,Cell.Barcode,DF_pANN,DF_classification,DF_pANN_quantile,Spliced.Counts,Spliced.Features,Unspliced.Counts,Unspliced.Features,Ambiguous.Counts,Ambiguous.Features
Cell.ID,,,,,,,,,,,,,,,,,,,,,
hft_w20_p3_r1_AAACCCAAGCTGCGAA,hft_w20_p3_r1,pcw20,HFT3,HFT,RNA v3,b2019_06,c16,1397,677,0.070866,...,AAACCCAAGCTGCGAA,0.086339,Singlet,0.356997,1063,544,67,54,94,69
hft_w20_p3_r1_AAACCCAAGGTAGTAT,hft_w20_p3_r1,pcw20,HFT3,HFT,RNA v3,b2019_06,c11,14338,4301,0.053200,...,AAACCCAAGGTAGTAT,0.325683,Singlet,0.821429,10339,3514,5437,2526,1431,669
hft_w20_p3_r1_AAACCCACAACTCCAA,hft_w20_p3_r1,pcw20,HFT3,HFT,RNA v3,b2019_06,c17,9260,3481,0.043511,...,AAACCCACAACTCCAA,0.397814,Doublet,0.984402,6494,2701,6860,2515,1095,669
hft_w20_p3_r1_AAACCCACATAGTCAC,hft_w20_p3_r1,pcw20,HFT3,HFT,RNA v3,b2019_06,c0,4025,1969,0.015396,...,AAACCCACATAGTCAC,0.076503,Singlet,0.314723,2655,1475,5875,2058,634,377
hft_w20_p3_r1_AAACCCAGTACAGGTG,hft_w20_p3_r1,pcw20,HFT3,HFT,RNA v3,b2019_06,c4,7131,2930,0.044690,...,AAACCCAGTACAGGTG,0.239344,Singlet,0.746356,5008,2228,6026,2106,909,556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hft_w16_p7_r2_TTTGTTGCAGCACCCA,hft_w16_p7_r2,pcw16,HFT7,HFT,RNA NG,b2020_03,c6,8532,3497,0.018749,...,TTTGTTGCAGCACCCA,0.389488,Doublet,0.961542,6226,2730,4194,2059,973,612
hft_w16_p7_r2_TTTGTTGCAGGCTACC,hft_w16_p7_r2,pcw16,HFT7,HFT,RNA NG,b2020_03,c2,6689,2393,0.045577,...,TTTGTTGCAGGCTACC,0.210916,Singlet,0.663941,4757,1877,3039,1362,677,396
hft_w16_p7_r2_TTTGTTGGTCGCTTAA,hft_w16_p7_r2,pcw16,HFT7,HFT,RNA NG,b2020_03,c2,3865,1799,0.035954,...,TTTGTTGGTCGCTTAA,0.074798,Singlet,0.306766,2808,1409,1436,832,431,273


In [22]:
# Meta data is including the information about patients.
# Put prefix to indicate it
meta = meta.add_prefix('Auth_')

### 2.4 Optimizing adata.X data type to uint16 
* optimized data type boosts the process

In [ ]:
# Check if adata.X contains non-integer data
# If the data is not big, check all
# If the data is too big, check random data
if adata.shape[0] <= 250000: 
    print('Checking complete adata')
    if np.equal(np.mod(adata.X.toarray(), 1), 0).all() != True:
        print('CAREFUL: non-integer matrix loaded!')
else:
    print ('Checking a subsample of adata')
    if np.equal(np.mod(sc.pp.subsample(adata, n_obs=250000, random_state=0, copy=True).X.toarray(), 1), 0).all() != True:
        print('CAREFUL: non-integer matrix loaded!')

In [ ]:
# Check if the maximum value in the data fits within the 'uint16' range (0 ~ 65535)
# 32,000 <- safe threshold
maxCount = csr_matrix.max(adata.X)
if maxCount < 32000: 
    print('Change X type to integer')
    adata.X = adata.X.astype('uint16') # Optimization to 'unit16'
    if maxCount != csr_matrix.max(adata.X):
        print('CAREFUL: max count value has changed!')
else: 
    print('X type not changed') # In this case, consider different data type

### 2.5 Build adata.obs

In [ ]:
if meta.shape[0] != adata.obs.shape[0]:
    print('CAREFUL: expression matrix and metadata size are not coherent!')
    print ('Metadata rows: ' + str(meta.shape[0]))

In [ ]:
adata.obs = adata.obs.join(meta, how='left', validate='one_to_one')

### Build adata.var

In [ ]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name","start_position", "end_position", "chromosome_name"]).set_index("ensembl_gene_id")

In [ ]:
adata.var = adata.var.join(annot, how='left', validate='one_to_one')
print(adata.var['external_gene_name'].isnull().sum())

In [ ]:
adata.var['ensg'] = adata.var.index.tolist()
adata.var['external_gene_name'] = adata.var['external_gene_name'].fillna(adata.var.ensg)

In [ ]:
#replace external gene name as index
adata.var.index = adata.var['external_gene_name']
adata.var.drop('external_gene_name', axis = 1, inplace = True)
adata.var_names_make_unique()

### Add additional fields to adata.obs

In [ ]:
adata.obs['dataset_id'] = 'Id0003'
adata.obs['sample_id'] = adata.obs['Auth_Sample.ID']
adata.obs['brain_region'] = 'cerebral_cortex'
adata.obs['age'] = adata.obs['Auth_Age'].str.replace('pcw', 'PCW_')
adata.obs['stage'] = 'prenatal'
adata.obs['batch_key'] = adata.obs['Auth_Batch'] + '_' + adata.obs['Auth_Assay'].str.replace(' ', '')

In [ ]:
Dict = {'c0': 'ExN_N5', 'c1': 'In_CGE', 'c2': 'ExN_N1',
        'c3': 'In_MGE', 'c4': 'ExN_N4', 'c5': 'ExN_N2', 
        'c6': 'RG_early', 'c7': 'ExN_N7', 'c8': 'CycProg', 
        'c9': 'ExN_N3', 'c10': 'RG_late', 'c11': 'GliaPg', 
        'c12': 'ExN_N6', 'c13': 'SubPlate', 'c14': 'IPC', 
        'c15': 'ExN_N8', 'c16': 'Microglia', 'c17': 'OPC_Oligo', 
        'c18': 'tRG', 'c19': 'Pericytes', 'c20': 'Endo',
        'c21': 'RBC', 'c22': 'VLMC'
       }

adata.obs['cell_label'] = adata.obs['Auth_seurat_clusters'].replace(Dict)

### Save

In [ ]:
if isspmatrix(adata.X) == False:
    adata.X = csr_matrix(adata.X)
    print('Converted adata.X to', type(adata.X))

In [ ]:
adata.write(output_file, compression='gzip')